## Example notebook to generate bactrack links for SuperSegger2 using Gurobi solver

In [1]:
from bactrack.core import compute_hierarchy
dir_phase = "/home/tlo/Documents/Data/bactrack_test2/xy0/phase/"
# chose hypermodel between Cellpose and Omnipose
# chose submodel under instruction of hypermodel
hier_arr = compute_hierarchy(dir_phase, submodel= 'bact_phase_omni')

#from bactrack.core import ModelEnum

# hypermodel = ModelEnum.OMNIPOSE
# hier_arr = compute_hierarchy(dir,hypermodel, submodel= 'bact_phase_omni')

2024-04-10 15:17:55,193 [INFO] TORCH GPU version not installed/working.
2024-04-10 15:17:55,194 [INFO] >>bact_phase_omni<< model set to be used
2024-04-10 15:17:55,195 [INFO] >>>> using CPU


  0%|          | 0/31 [00:00<?, ?it/s]

2024-04-10 15:17:55,487 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:55,903 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:56,241 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:56,821 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:57,223 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:57,618 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:57,949 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:58,323 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:58,688 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:59,011 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:59,352 [INFO] Running on tiles. Now normalizing each tile separately.
2024-04-10 15:17:59,678 [INFO] Running on t

100%|███████████████████████████████████████████| 31/31 [00:01<00:00, 15.56it/s]

2024-04-10 15:18:08,673 [INFO] Segmentation hierarchy builded.
2024-04-10 15:18:08,713 [INFO] Labeled feature of each segmentation in hierarchy.


In [2]:
from bactrack.tracking import OverlapWeight, IOUWeight, DistanceWeight
w =  DistanceWeight(hier_arr)

2024-04-10 15:18:08,728 [INFO] Weight function start computing weight matrix
2024-04-10 15:18:08,770 [INFO] Weight matrix computed, time consuming:0.04205918312072754 sec


In [3]:
#   MIPSolver => python_mip repo: it includes two solvers: CBC and Gurobi
#   ScipySolver => scipy.optimize.milp: it include HiGHS solvers

# MIPSolver require conda environment and additional setup, so it not always avilable, you could use 
# ScipySolver instead
from bactrack.tracking import MIPSolver, ScipySolver

# do solver = ScipySolver(w.weight_matrix, hier_arr, mask_penalty= w.mask_penalty)
# if you want include a mask_penalty calculated by omnipose/cellpose mask threshold + flow threshold
solver = MIPSolver(w.weight_matrix, hier_arr)

# n is a list of all selected segementation
# e is a scipy sparse matrix mark the edge be selected
n, e = solver.solve()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-12
2024-04-10 15:18:08,863 [INFO] Setting up MIP problem
2024-04-10 15:18:08,891 [INFO] MIP problem set up: time used 0.02681565284729004 sec
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-4790 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 628 rows, 1503 columns and 3204 nonzeros
Model fingerprint: 0xb934f3e3
Variable types: 0 continuous, 1503 integer (1327 binary)
Coefficient statistics:
  Matrix range     [2e-03, 4e+09]
  Objective range  [5e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
         Consider reformulating model or setting Num

In [4]:
from bactrack import io
# m is a mask array including all output masks
# e is a edge dataframe
# also selected cell in  hier_arr be marked in this function
m, e = io.format_output(hier_arr, n, e)
# now n is  a node dataframe
n = io.hiers_to_df(hier_arr)

In [5]:
# merge
import pandas as pd

merged_df = pd.merge(e, n.add_suffix('_source'), left_on='Source Index', right_on='index_source', how='left')
merged_df = pd.merge(merged_df, n.add_suffix('_target'), left_on='Target Index', right_on='index_target', how='left')

In [6]:
selected_df = merged_df[['frame_source','label_source', 'frame_target', 'label_target']]
print(selected_df)

     frame_source  label_source  frame_target  label_target
0               0           1.0             1           2.0
1               0           3.0             1           1.0
2               0           3.0             1           3.0
3               1           1.0             2           1.0
4               1           2.0             2           2.0
..            ...           ...           ...           ...
129            29           2.0            30           1.0
130            29           2.0            30           4.0
131            29           3.0            30           5.0
132            29           4.0            30           2.0
133            29           4.0            30           3.0

[134 rows x 4 columns]


In [7]:
# export links to csv
import os

xydir = os.path.abspath(os.path.join(dir_phase, os.pardir)) #get xydir path
bactrackdir = os.path.join(xydir, "bactrackfiles")
os.makedirs(bactrackdir, exist_ok=True)  #make dir in xydir for bactrack file
merged_df.to_csv(os.path.join(bactrackdir, "mergedlinks.csv")) #save full merged links file 
selected_df = merged_df[['frame_source','label_source', 'frame_target', 'label_target']]
selected_df.to_csv(os.path.join(bactrackdir, "superseggerlinks.csv")) #save links file relevant for supersegger

In [8]:
# export bactrack masks

maskdir = os.path.join(xydir,"bactrackmasks")
os.makedirs(maskdir, exist_ok=True)  #make dir for bactrack file



io.store_mask_arr(m, maskdir)